<a href="https://colab.research.google.com/github/nikhilareddyp9/Music_Generation_-RNN/blob/main/Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install  music21

In [ ]:
import sys
import re 
import numpy 
import pandas

import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from keras.utils import np_utils

from music21 import converter, instrument, note, chord, stream


In [ ]:
songs = glob('Jazz/*.mid')
songs = songs[:3]

In [ ]:
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth
!pip install pretty_midi

In [ ]:
import pretty_midi as musical_inf

import datetime
import glob
import numpy
import pathlib
import fluidsynth
import pandas
import seaborn as sns
import collections
import tensorflow


from IPython import display as show
from matplotlib import pyplot
from typing import Optional


In [ ]:
randomseed = 42
tensorflow.random.set_seed(randomseed)
numpy.random.seed(randomseed)


In [ ]:
# Sampling rate for audio playback
instance_hertz = 16000

import urllib.request
import zipfile
import os as pull

directory = 'data/maestro-v2.0.0'
if not pull.path.exists(directory):
    pull.makedirs(directory)
    url = 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
    urllib.request.urlretrieve(url, f'{directory}/maestro-v2.0.0-midi.zip')
    with zipfile.ZipFile(f'{directory}/maestro-v2.0.0-midi.zip', 'r') as zip_ref:
        zip_ref.extractall(directory)

folders = []
for I, _, cases in pull.walk(directory):
    for z in cases:
        if z.endswith('.mid') or z.endswith('.midi'):
            folders.append(pull.path.join(I, z))

print('Number of music files scrutinized:', len(folders))

instancefile = folders[1]
print(instancefile)
prettyMIDI = musical_inf.PrettyMIDI(instancefile)

def show_phonic(prettyMIDI: musical_inf.PrettyMIDI, seconds=35):
    duration = prettyMIDI.get_end_time()
    if seconds > duration:
        seconds = duration
    samples = int(seconds * instance_hertz)
    waveform = prettyMIDI.fluidsynth(fs=instance_hertz)
    waveform_short = waveform[:samples]
    return show.Audio(waveform_short, rate=instance_hertz)

show_phonic(prettyMIDI)

instruments_used = prettyMIDI.instruments
print('Number of instruments:', len(instruments_used))
if len(instruments_used) > 0:
    instrument = instruments_used[0]
    toolname= musical_inf.program_to_instrument_name(instrument.program)
    print('Tool used name:', toolname)
else:
    print('No Tools found in the PrettyMIDI object.')


In [ ]:
!pip install --upgrade music21

In [ ]:
for k_pole, score in enumerate(instrument.notes[:10]):
  notesname = musical_inf.note_number_to_name(score.pitch)
  timepernote = float(score.end - score.start)
  print(str(k_pole) +': '+'Note= '+str(notesname)+' Pitch= '+str(score.pitch)+' Time= '+str(round(timepernote,3)))

def convert_midis2notes(midi: str) -> pandas.DataFrame:
    prettyMIDI = musical_inf.PrettyMIDI(midi)
    tool = prettyMIDI.instruments[0]
    notes = collections.defaultdict(list)

    sorted_notes = sorted(tool.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start

    for note in sorted_notes:
        start = note.start
        end = note.end
        notes['pitch'].append(note.pitch)
        notes['start'].append(start)
        notes['end'].append(end)
        notes['step'].append(start - prev_start)
        notes['duration'].append(end - start)
        prev_start = start

    return pandas.DataFrame({name: numpy.array(value) for name, value in notes.items()})

rawtranscription = convert_midis2notes(instancefile)
rawtranscription.head()

def note_numbers2names(note_numbers: numpy.ndarray) -> numpy.ndarray:
    return numpy.vectorize(musical_inf.note_number_to_name)(note_numbers)

transcriptionsample = note_numbers2names(rawtranscription['pitch'])[:10]
transcriptionsample[:10]

def show_piano_roll(df_notes: pandas.DataFrame, max_notes: Optional[int] = None):
    if max_notes:
        plot_title,notes_to_plot = f'Begin {max_notes} Transcription',df_notes.iloc[:max_notes]
    else:
        plot_title,notes_to_plot  = f'Full audio',df_notes
    pyplot.figure(figsize=(21, 5))
    pitch_array = numpy.stack([notes_to_plot['pitch'], notes_to_plot['pitch']], axis=0)
    time_array = numpy.stack([notes_to_plot['start'], notes_to_plot['end']], axis=0)
    pyplot.plot(time_array[:, :max_notes], pitch_array[:, :max_notes], color="r", marker="|")
    pyplot.xlabel('Time in secs')
    pyplot.ylabel('Pitch')
    _ = pyplot.title(plot_title)


In [ ]:
show_piano_roll(rawtranscription, max_notes=100)

show_piano_roll(rawtranscription)


In [ ]:
def plot_note_distributions(note_df: pandas.DataFrame, fall: float = 2.5):
    fig, axis = pyplot.subplots(1, 3, figsize=(15,5))


    axis[0].hist(note_df['pitch'], bins=19)
    axis[0].set_xlabel('Pitch')
    axis[0].set_ylabel('Count')

    max_step = numpy.percentile(note_df['step'], 100 - fall)
    axis[1].hist(note_df['step'], bins=numpy.linspace(0, max_step, 20))
    axis[1].set_xlabel('Step')
    axis[1].set_ylabel('Count')

    max_duration = numpy.percentile(note_df['duration'], 100 - fall)
    axis[2].hist(note_df['duration'], bins=numpy.linspace(0, max_duration, 20))
    axis[2].set_xlabel('Duration')
    axis[2].set_ylabel('Count')

    pyplot.show()


In [ ]:
plot_note_distributions(rawtranscription)

In [ ]:
def create_midi_file(notes_df: pandas.DataFrame, result: str, toolused: str, speed: int = 100) -> musical_inf.PrettyMIDI:
    prettyMIDI = musical_inf.PrettyMIDI()
    instrument_program = musical_inf.instrument_name_to_program(toolused)
    instrument_obj = musical_inf.Instrument(program=instrument_program)   
    prev_note_end = 0
    for idx, I in notes_df.iterrows():
        start_time,pitch = prev_note_end + I['step'],int(I['pitch'])
        end_time = start_time + I['duration']
        new_note = musical_inf.Note(velocity=speed,pitch=pitch,start=start_time,end=end_time)
        instrument_obj.notes.append(new_note)
        prev_note_end = end_time
    prettyMIDI.instruments.append(instrument_obj)
    prettyMIDI.write(result)    
    return prettyMIDI


In [ ]:
instancefile = 'example.midi'
example_prettyMIDI = create_midi_file(rawtranscription, result=instancefile, toolused=toolname)

show_phonic(example_prettyMIDI)


In [ ]:
TO_PARSE = 5


transcription_array = []
for file_path in folders[:TO_PARSE]:
    transcription = convert_midis2notes(file_path)
    transcription_array.append(transcription)


transcription_df = pandas.concat(transcription_array)
num_transcription = len(transcription_df)
print(f"Count of transcription: {num_transcription}")


In [ ]:
feature_names = ['pitch', 'step', 'duration']
features = numpy.stack([transcription_df[feature] for feature in feature_names], axis=1)
transcription_ds = tensorflow.data.Dataset.from_tensor_slices(features)


print(f"Dataset element spec: {transcription_ds.element_spec}")


In [ ]:
seriessize = 25
vocablen = 128

def generate_transcription(processingfile: tensorflow.data.Dataset, seriessize: int, vocablen: int = 128) -> tensorflow.data.Dataset:
    """Returns a TensorFlow dataset of sequence and label examples."""
    seriessize = seriessize + 1


    Xfile = processingfile.window(seriessize, shift=1, stride=1, drop_remainder=True)


    scruten = lambda x: x.batch(seriessize, drop_remainder=True)
    sequences = Xfile.flat_map(scruten)


    def normalize(y):
        y = y / [vocablen, 1.0, 1.0]
        return y


    def divide_transcription(series):
        data = series[:-1]
        Dnames = series[-1]
        names = {key: Dnames[i] for i, key in enumerate(feature_names)}

        return normalize(data), names


    return sequences.map(divide_transcription, num_parallel_calls=tensorflow.data.AUTOTUNE)
dl_int = generate_transcription(transcription_ds, seriessize, vocablen)
dl_int.element_spec

for sequence, tl in dl_int.take(1):
    print('Arrangement of the series:', sequence.shape)
    print('Highs in the sequence (15):', sequence[0: 15])
    print()
    print('target label:', tl)


In [ ]:
size_of_batch = 64
size_of_buffer = num_transcription - seriessize  
ds_train = (dl_int
            .shuffle(size_of_buffer)
            .batch(size_of_batch, drop_remainder=True)
            .cache()
            .prefetch(tensorflow.data.experimental.AUTOTUNE))
ds_train.element_spec

def calculate_meansquareerror_pp(y_true_labels: tensorflow.Tensor, y_predicted_labels: tensorflow.Tensor):
  """Calculates mean squared error loss with positive pressure."""
  meansqureerror_loss = (y_true_labels - y_predicted_labels) ** 2
  pp = 10 * tensorflow.maximum(-y_predicted_labels, 0.0)
  return tensorflow.reduce_mean(meansqureerror_loss + pp)


In [ ]:
seq = (seriessize, 3)
rate_of_learn_value = 0.005

inputlayer = tensorflow.keras.Input(seq)
lstm_layer = tensorflow.keras.layers.LSTM(128)(inputlayer)

output_layers = {
  'pitch': tensorflow.keras.layers.Dense(128, name='pitch')(lstm_layer),
  'step': tensorflow.keras.layers.Dense(1, name='step')(lstm_layer),
  'duration': tensorflow.keras.layers.Dense(1, name='duration')(lstm_layer),
}

mod_generated = tensorflow.keras.Model(inputlayer, output_layers)

loss_fn = {
      'pitch': tensorflow.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True),
      'step': calculate_meansquareerror_pp,
      'duration': calculate_meansquareerror_pp,
}

shaper = tensorflow.keras.optimizers.Adam(learning_rate=rate_of_learn_value)

mod_generated.compile(loss=loss_fn, optimizer=shaper)

mod_generated.summary()


In [ ]:
losses = mod_generated.evaluate(ds_train, return_dict=True)
mod_generated.compile(loss=loss_fn,loss_weights={'pitch': 0.05,'step': 1.0,'duration':1.0,},optimizer=shaper,)

CB = [
    tensorflow.keras.callbacks.ModelCheckpoint(
        filepath='./training_checkpoints/ckpt_{epoch}',
        save_weights_only=True),
    tensorflow.keras.callbacks.EarlyStopping(
        verbose=1,
        monitor='loss',
        restore_best_weights=True,
        patience=5),
]


In [ ]:
# Commented out IPython magic to ensure Python compatibility.
%%time
epochs = 50
result = mod_generated.fit(ds_train,epochs=epochs,callbacks=CB,)

pyplot.plot(result.epoch,result.history['loss'], label='total loss')
pyplot.show()


In [ ]:
def predict_next_note(preds: numpy.ndarray, 
                  kmodel: tensorflow.keras.Model, 
                  newtemp: float = 1.0) -> tuple[int, float, float]:
    """Generates a note as a tuple of (pitch, step, duration), using a trained sequence model."""

    assert newtemp > 0

    data = tensorflow.expand_dims(preds, 0)

    outputs_notes = kmodel.predict(data)
    logits = outputs_notes['pitch']
    stepp = outputs_notes['step']
    t_insecs = outputs_notes['duration']
 
    logits /= newtemp
    ptch = tensorflow.random.categorical(logits, num_samples=1)
    ptch = tensorflow.squeeze(ptch, axis=-1)
    t_time = tensorflow.squeeze(t_insecs, axis=-1)
    stepp = tensorflow.squeeze(stepp, axis=-1)


    stepp = tensorflow.maximum(0, stepp)
    t_insecs = tensorflow.maximum(0, t_time)

    return int(ptch), float(stepp), float(t_insecs)


In [ ]:
new_degree = 2.0
num_of_preds = 120

sampnote = numpy.stack([rawtranscription[I] for I in feature_names], axis=1)


data_trans = (
    sampnote[:seriessize] / numpy.array([vocablen, 1, 1]))

new_notes = []
pbegin = 0
for _ in range(num_of_preds):
  pitch, step, duration = predict_next_note(data_trans, mod_generated, new_degree)
  begin_time = pbegin + step
  final_time = begin_time + duration
  data = (pitch, step, duration)
  new_notes.append((*data, begin_time, final_time))
  data_trans = numpy.delete(data_trans, 0, axis=0)
  data_trans = numpy.append(data_trans, numpy.expand_dims(data, 0), axis=0)
  pbegin = begin_time


In [ ]:
new_notes_df = pandas.DataFrame(new_notes, columns=(*feature_names, 'start', 'end'))

new_notes_df.head(10)


In [ ]:
resultfile = 'output.mid'
resultMIDI = create_midi_file(
    new_notes_df, result=resultfile, toolused=toolname)
show_phonic(resultMIDI)

from google.colab import files
files.download(resultfile)


In [ ]:
show_piano_roll(new_notes_df)

In [ ]:
plot_note_distributions(new_notes_df)